In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import csv
import re
import string
import unicodedata
from bpe_trainer import BPE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/nlp_exam

/content/drive/.shortcut-targets-by-id/1TLubliFZe2hWlPTQRh7pkAkvkWbfAB9W/nlp_exam


This is the first file to run, we take a *txt* file (https://github.com/kite1988/nus-sms-corpus) containing more then 55.000 messages. We will create a *.csv* file that contains just the messages (because we are definelty not interested about all the other informations inside the *.sql* file), we will then use this new file as training for the BPE (Byte Pair Encoding)

In [ ]:
with open("trainBPE/fortraining_smsCorpus_en_2015.03.09_all.txt", "r") as file:
    content = file.read()

split_text = content.split("),(")


with open("trainBPE/BPE_trainer.csv", "w+") as file:
    for text in split_text:
        file.write(text + "\n")

Now we load the BPE_trainer.csv file and we remove all the columns except the one that contains the messages.

In [ ]:
df = pd.read_csv("trainBPE/BPE_trainer.csv", header=None, error_bad_lines=False)
df = df[df.apply(lambda row: len(row) == 19, axis=1)]
df.to_csv("trainBPE/BPE_trainer.csv", index=False)

Streaming output truncated to the last 5000 lines.
Skipping line 14007: expected 19 fields, saw 20
Skipping line 14008: expected 19 fields, saw 20
Skipping line 14009: expected 19 fields, saw 20
Skipping line 14010: expected 19 fields, saw 20
Skipping line 14011: expected 19 fields, saw 20
Skipping line 14012: expected 19 fields, saw 20
Skipping line 14013: expected 19 fields, saw 20
Skipping line 14014: expected 19 fields, saw 20
Skipping line 14015: expected 19 fields, saw 20
Skipping line 14016: expected 19 fields, saw 20
Skipping line 14017: expected 19 fields, saw 21
Skipping line 14018: expected 19 fields, saw 20
Skipping line 14019: expected 19 fields, saw 21
Skipping line 14020: expected 19 fields, saw 21
Skipping line 14021: expected 19 fields, saw 20
Skipping line 14022: expected 19 fields, saw 20
Skipping line 14023: expected 19 fields, saw 20
Skipping line 14024: expected 19 fields, saw 20
Skipping line 14025: expected 19 fields, saw 20
Skipping line 14026: expected 19 fiel

In [ ]:
print(df.shape)

(48069, 19)


In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,('79780a9dbe83fd1e5dd2bd2543e7da2a','0ffc7585148560b7520931d354c00a9b','2010.10.24 11:59','2010/11','SMS Export','K','yes','India','21-25','Multi-tap','3 to 5 years','More than 50 SMS daily','Nokia','Tao Chen','unknown','unknown','en','Tiruppur',1
1,'79780a9dbe83fd1e5dd2bd2543e7da2a','ce3e0c80307aea9b4f656890d19735b3','2010.10.28 11:53','2010/11','SMS Export','Studying?','yes','India','21-25','Multi-tap','3 to 5 years','More than 50 SMS daily','Nokia','Tao Chen','unknown','unknown','en','Tiruppur',2
2,'79780a9dbe83fd1e5dd2bd2543e7da2a','0ffc7585148560b7520931d354c00a9b','2010.10.26 22:21','2010/11','SMS Export','Vch photo','yes','India','21-25','Multi-tap','3 to 5 years','More than 50 SMS daily','Nokia','Tao Chen','unknown','unknown','en','Tiruppur',3
3,'79780a9dbe83fd1e5dd2bd2543e7da2a','0ffc7585148560b7520931d354c00a9b','2010.10.25 20:27','2010/11','SMS Export','K:-)ya i hav to finish','yes','India','21-25','Multi-tap','3 to 5 years','More than 50 SMS daily','Nokia','Tao Chen','unknown','unknown','en','Tiruppur',4
4,'79780a9dbe83fd1e5dd2bd2543e7da2a','0ffc7585148560b7520931d354c00a9b','2010.10.25 08:33','2010/11','SMS Export','K','yes','India','21-25','Multi-tap','3 to 5 years','More than 50 SMS daily','Nokia','Tao Chen','unknown','unknown','en','Tiruppur',5


In [ ]:
# remove all the columns except the 5th one
df = df.iloc[:, [5]].rename(columns={5: "messages"})
print(df)

                                                messages
0                                                    'K'
1                                            'Studying?'
2                                            'Vch photo'
3                               'K:-)ya i hav to finish'
4                                                    'K'
...                                                  ...
48064  'Hmmm... Thk i usually book on wkends... Depen...
48065  'ask them got any sms messages to gif me lei.....
48066                          'We r near coca oredi...'
48067  'hall Eleven. Got lectures le mah.èn forget ab...
48068  'I Bring for u. I can not promise u 100% to wi...

[48069 rows x 1 columns]


In [ ]:
# find duplicates...
df.duplicated().sum()

6819

In [ ]:
# ...and remove duplicates
df = df.drop_duplicates(keep="first")
df.duplicated().sum()

0

In [ ]:
df.head()

,messages
0,'K'
1,'Studying?'
2,'Vch photo'
3,'K:-)ya i hav to finish'
5,'One senioq akka'


In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\x00-\x7F]+', '', text) # remove non ascii chracter
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    #text = re.sub('\[.*?\]', '', text)
    #text = re.sub('https?://\S+|www\.\S+', '', text)
    #text = re.sub('<.*?>+', '', text)
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)

    return text

In [ ]:
df['messages'] = df['messages'].apply(clean_text)
df.head()

,messages
0,k
1,studying
2,vch photo
3,kya i hav to finish
5,one senioq akka


In [ ]:
output_file_path = 'trainBPE/txt_to_train_BPE.csv'

with open(output_file_path, 'w') as output_file:
    for message in df["messages"]:
        output_file.write(str(message) + '\n')


In [ ]:
df_csv = pd.read_csv(output_file_path, header=None, names=["messages"])
print(df_csv.head())

              messages
0                    k
1             studying
2            vch photo
3  kya i hav to finish
4      one senioq akka


# Load spam.csv

In [ ]:
spamdf = pd.read_csv("spam.csv", encoding='latin1')

In [ ]:
spamdf.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
spamdf = spamdf.dropna(how="any", axis=1)
spamdf.columns = ['target', 'messages']
spamdf.head()

,target,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
spamdf['messages'] = spamdf['messages'].apply(clean_text)
spamdf.head()

,target,messages
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


## BPE

In [ ]:
train_file_path = 'trainBPE/txt_to_train_BPE.csv'  # should be a txt

bpe = BPE(train_file_path)
bpe.train(1000)

Training finished


In [ ]:
output_file_path = 'trainBPE/full_tokenization.txt'

with open(output_file_path, 'w') as output_file:
    for index, row in spamdf.iterrows():
        if len(row) >= 2:
            text = row[1].strip()

            words = text.split()

            tokens = [bpe.tokenize(word) for word in words]

            output_file.write(str(tokens) + '\n')


# Create the emebdding

In [ ]:
token_file = open("trainBPE/full_tokenization.txt", 'r')
for _ in range(5):
    line = token_file.readline()
    print(line)

[['go</w>'], ['until</w>'], ['ju', 'r', 'ong</w>'], ['po', 'int</w>'], ['cra', 'zy</w>'], ['a', 'v', 'ai', 'la', 'ble</w>'], ['only</w>'], ['in</w>'], ['bu', 'g', 'is</w>'], ['n</w>'], ['great</w>'], ['wor', 'ld</w>'], ['la</w>'], ['e</w>'], ['bu', 'ffe', 't</w>'], ['c', 'ine</w>'], ['there</w>'], ['got</w>'], ['a', 'more</w>'], ['wat</w>']]

[['ok</w>'], ['lar</w>'], ['j', 'ok', 'ing</w>'], ['wif</w>'], ['u</w>'], ['on', 'i</w>']]

[['free</w>'], ['en', 'try</w>'], ['in</w>'], ['a</w>'], ['w', 'k', 'ly</w>'], ['com', 'p</w>'], ['to</w>'], ['w', 'in</w>'], ['f', 'a</w>'], ['c', 'up</w>'], ['fin', 'al</w>'], ['t', 'k', 'ts</w>'], ['may</w>'], ['te', 'xt</w>'], ['f', 'a</w>'], ['to</w>'], ['to</w>'], ['rece', 'ive</w>'], ['en', 'try</w>'], ['que', 'sti', 'on', 'st', 'd</w>'], ['t', 'xt</w>'], ['ra', 'te', 't', 'cs</w>'], ['app', 'ly</w>']]

[['u</w>'], ['dun</w>'], ['say</w>'], ['so</w>'], ['early</w>'], ['hor</w>'], ['u</w>'], ['c</w>'], ['already</w>'], ['then</w>'], ['say</w>']]

[['n

In [ ]:
def create_mapping(txt_file_in, txt_file_out):
    mapping = {}
    current_number = 1

    with open(txt_file_in, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip().replace('[', '').replace(']', '').replace(',', '')
            words = line.split()

            for word in words:
                word = word.replace("'", "")
                if word not in mapping:
                    mapping[word] = current_number
                    current_number += 1

    embedding_size = 5
    embedding = nn.Embedding(len(mapping) + 1, embedding_size)
    embedding_vectors = torch.randn(len(mapping) + 1, embedding_size)

    with open(txt_file_out, 'w') as file:
        for word, number in mapping.items():
            embedding_vector = embedding_vectors[number]
            file.write(f"{word} {' '.join(map(str, embedding_vector.tolist()))}\n")

txt_file_in = 'trainBPE/full_tokenization.txt'
txt_file_out = 'dict_spam_embedding.txt'

create_mapping(txt_file_in, txt_file_out)

## Test emebdding

In [ ]:
token_file = open("dict_spam_embedding.txt", 'r')

for _ in range(5):
    line = token_file.readline()
    print(line)

go</w> -1.0736626386642456 0.806194007396698 -0.2725912034511566 -0.17693957686424255 -1.2118901014328003

until</w> 1.825268268585205 -1.2477036714553833 -0.630555272102356 -0.8589311838150024 0.6999149918556213

ju -0.05993494391441345 -0.14328832924365997 -0.7662547826766968 0.4969610869884491 0.18791404366493225

r 0.7064007520675659 -0.7704875469207764 -1.1224980354309082 -0.6367014050483704 0.25777485966682434

ong</w> -0.5498090386390686 -1.0631375312805176 -1.0584694147109985 -0.845133364200592 0.05983300507068634



In [ ]:
file_name = "dict_spam_embedding.txt"

with open(file_name, 'r') as file:
    content = file.read()
    print(content)

go</w> -1.0736626386642456 0.806194007396698 -0.2725912034511566 -0.17693957686424255 -1.2118901014328003
until</w> 1.825268268585205 -1.2477036714553833 -0.630555272102356 -0.8589311838150024 0.6999149918556213
ju -0.05993494391441345 -0.14328832924365997 -0.7662547826766968 0.4969610869884491 0.18791404366493225
r 0.7064007520675659 -0.7704875469207764 -1.1224980354309082 -0.6367014050483704 0.25777485966682434
ong</w> -0.5498090386390686 -1.0631375312805176 -1.0584694147109985 -0.845133364200592 0.05983300507068634
po -0.412460595369339 0.9278625845909119 -0.3769344389438629 -1.3919068574905396 0.23493340611457825
int</w> 0.8919991850852966 -0.8658770322799683 1.2798709869384766 -1.0895315408706665 0.730492889881134
cra 1.5086554288864136 0.26055553555488586 1.1778188943862915 -1.0528490543365479 -1.2617219686508179
zy</w> -1.7746080160140991 -0.025748075917363167 -1.2018448114395142 -1.7776877880096436 -0.14595246315002441
a 0.8659927845001221 -0.24791787564754486 -0.50450009107589